In [ ]:
Life Cycle Assessment

For a sustainable bioeconomy, it is critical to assess the environmental impact of a production process. This chapter demonstrates BioSTEAM's life cycle assessment (LCA) capabilities through a cradle-to-gate LCA of a sugarcane biorefinery that produces electricity and ethanol. We focus on green-house gas (GHG) emissions because it is central to policy implementation.

In [ ]:
import biosteam as bst


from biorefineries import sugarcane as sc


sc.load()


sys = sc.sugarcane_sys


feeds = sys.feeds


material_cradle_to_gate_key = ('GWP100', 'cradle_to_gate')


electricity_key = ('GWP100', 'cradle_to_gate')


for feed in sys.feeds:


    # Assume all other feeds are negligible


    feed.characterization_factors[material_cradle_to_gate_key] = 0.





# Set non-negligible characterization factors


# sc.sugarcane.characterization_factors[material_cradle_to_gate_key] = 0.044 # https://www.osti.gov/servlets/purl/1337146


dw = (sc.sugarcane.F_mass-sc.sugarcane.imass['H2O'])/sc.sugarcane.F_mass


sc.sugarcane.characterization_factors[material_cradle_to_gate_key] = 0.044*dw # https://www.osti.gov/servlets/purl/1337146


bst.PowerUtility.characterization_factors[electricity_key] = 0.48 # https://greet.es.anl.gov/





# Get net electricity data


net_electricity = bst.PowerUtility.sum([i.power_utility for i in sys.units])


GWP_material = sum([s.get_impact(material_cradle_to_gate_key) for s in sys.feeds])


biogenic_CO2 = sc.sugarcane.get_atomic_flow('C')*44 # in kg CO2/hr


GWP_direct_emissions = sum([i.imass['CO2'] for i in sys.products if 'CO2' in i.chemicals]) - biogenic_CO2


GWP_electricity_production = net_electricity.get_impact(production_key=electricity_key)


GWP_total = GWP_material + GWP_direct_emissions + GWP_electricity_production # kg CO2 eq. / hr


GWP_sugarcane_ethanol = GWP_total / sc.ethanol.F_mass # kg CO2 eq. / kg sugarcane ethanol


print(GWP_sugarcane_ethanol)